In [16]:
import surprise
from surprise import SVD
from surprise import Dataset, Reader 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [2]:
# 내장 데이터를 로드하고 학습과 테스트 데이터로 분리
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\Taylor/.surprise_data/ml-100k


In [3]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [5]:
# 추천 행렬 분해 알고리즘으로 SVD객체를 생성하고 학습수행
algo = SVD()
algo.fit(trainset)

In [6]:
# 테스트 데이터 세트에 예상 평점 데이터 예측.
# test()메소드 호출 시 Prediction 객체의 리스트로 평점 예측 데이터 반환
predictions = algo.test(testset)
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.2363875907017334, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.778360968691157, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.072222831503929, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.765464122039414, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.3455916832333488, details={'was_impossible': False})]

In [7]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.2363875907017334),
 ('882', '291', 3.778360968691157),
 ('535', '507', 4.072222831503929)]

In [8]:
# predict()메소드는 개별 사용자,아이템에 대한 예측 평점 정보를 반환
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.22   {'was_impossible': False}


In [9]:
# 반환된 Prediction의 리스트 객체를 기반으로 RMSE 평가
accuracy.rmse(predictions)

RMSE: 0.9500


0.9499809225883025

# Surprise 주요 모듈 소개

In [11]:
import pandas as pd

# csv 파일로 사용자 평점 데이터 생성
ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('../data/ml-latest-small/ratings_noh.csv', index=False, header=False)

In [13]:
from surprise import Reader

# Reader클래스로 파일의 포맷팅 지정하고 Dataset의 load_from_file()을 이용하여 데이터셋 로딩
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('../data/ml-latest-small/ratings_noh.csv',reader=reader)

In [15]:
#학습과 테스트 데이터 세트로 분할하고 SVD로 학습후 테스트데이터 평점 예측 후 RMSE평가
rainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정
# n_factors = latent factor의 수
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 1.1481


1.1481498636769425

In [17]:
# 판다스 DataFrame기반에서 동일하게 재 수행
ratings = pd.read_csv('../data/ml-latest-small/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [20]:
# 교차 검증(Cross Validation)과 하이퍼 파라미터 튜닝
# cross_validate()를 이용한 교차 검증
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('../data/ml-latest-small/ratings.csv') # reading data in pandas df
# 최소 0.5, 최대 5.0
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8719  0.8708  0.8743  0.8691  0.8785  0.8729  0.0032  
MAE (testset)     0.6679  0.6713  0.6736  0.6690  0.6745  0.6713  0.0025  
Fit time          3.30    3.47    3.32    3.27    3.28    3.33    0.07    
Test time         0.09    0.09    0.16    0.09    0.16    0.12    0.03    


{'test_rmse': array([0.87194153, 0.8708307 , 0.87430516, 0.86912054, 0.878484  ]),
 'test_mae': array([0.66789212, 0.67131845, 0.67360961, 0.66899473, 0.67448613]),
 'fit_time': (3.299994468688965,
  3.4677090644836426,
  3.3246428966522217,
  3.2727997303009033,
  3.2762093544006348),
 'test_time': (0.09307456016540527,
  0.09252429008483887,
  0.1623997688293457,
  0.09316420555114746,
  0.15971875190734863)}

In [21]:
# GridSearchCV 이용
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8773009903186489
{'n_epochs': 20, 'n_factors': 50}
